In [2]:
import time
import datetime
import csv
import sys
import pandas as pd
import sys
import os
import re
import numpy as np
import json
import string
import glob

## Step 1: Get list of Submission IDs we want to look at

All of our submissions are stored in the "comments" directory, and the IDs are stored in the filenames (see getCommentThreads.py)

In [3]:

commentFiles = glob.glob('comments/*.json', recursive=True)
# deltaIDs is a list of all the thread IDs we want to look at
deltaIDs = []
for f in commentFiles:
    deltaIDs.append(f[-11:-5])

# Step 2: reading in categories (for election data only)

In [11]:
with open('trump_submissions.json') as json_file:  
    data = json.load(json_file)["data"]
with open('clinton_submissions.json') as json_file:  
     data.extend(json.load(json_file)["data"])

In [3]:
df_categories = pd.read_csv("categories.csv")

In [22]:
l1 = df_categories["Supports Clinton / Against Trump"].values

In [23]:
l2 = df_categories["Supports Trump / Against Clinton"].values

In [24]:
l3 = df_categories["Other Candidate/Nomination"].values
l4 = df_categories["Other"].values

# Step 3: Write only important data to .csv

In [1]:
# This function writes out submissions to CSV
def writeSubmissionsToCSV(data, f_out_delta, f_out_other, deltaIDs):
    LENGTH = len(data)
    print("Length of input: ", LENGTH)
    deltaSubmissions = []
    otherSubmissions = []
    for i in range(LENGTH):
        # we want title, id and submission text to write into a csv file for quanteda purposes
        title = data[i]["title"]
        title = (title.split())
        title.pop(0)
        final_title = ' '.join(title)
        curr_id = data[i]["id"]
        # get and clean submission
        try:
            urlFree = re.sub(r"http\S+", "",  ' '.join(data[i]["selftext"].split()))
        except:
            continue
        submission_text = urlFree
        if submission_text != "[removed]":
            cat_name = ""
            if curr_id in deltaIDs:
                if curr_id in l1:
                    cat_name = "Supports Clinton"
                elif curr_id in l2:
                    cat_name = "Supports Trump"
                elif curr_id in l3:
                    cat_name = "Other Candidate"
                else:
                    cat_name = "Other"
                deltaSubmissions.append([curr_id, final_title, "delta", cat_name, submission_text])
            else:
                otherSubmissions.append([curr_id, final_title, submission_text, "none"])
    df = pd.DataFrame(deltaSubmissions, columns=["id", "title",  "deltaBool",  "category", "selftext"])

    dfOther = pd.DataFrame(otherSubmissions, columns=["id", "title", "selftext", "deltaBool"])
    df.to_csv(f_out_delta, index=False)
    dfOther.to_csv(f_out_other, index=False)

In [ ]:
df = writeSubmissionsToCSV(data, "delta_submissions.csv", "other_submissions.csv", deltaIDs)